In [ ]:
%pylab notebook

# Evaluate the results of anisotropic training

## Fetch the CMP data to compare against

In [ ]:
import os
import sys
import numpy as np
from glob import glob

eval_txt = 'data/training/independant_12_layers/fold_01/eval.txt'
eval_files = [fn.strip() for fn in open(eval_txt).readlines()]
print "Found {} eval files".format(len(eval_files))

cmp_eval_files = [f for f in eval_files if '/cmp/' in f]
print "Found", len(cmp_eval_files), "from the CMP dataset", len(eval_files)-len(cmp_eval_files), "are our own"

In [ ]:
from pyfacades.util.metrics import Metrics

In [ ]:
!nvidia-smi

In [ ]:
import caffe
#caffe.set_mode_cpu()
caffe.set_mode_gpu()
caffe.set_device(0)

In [ ]:
import skimage.io
from skimage.transform import warp, ProjectiveTransform

In [ ]:
from pyfacades.rectify import Homography as AffaraRectifier

In [ ]:
from pyfacades.util import Metrics, softmax, channels_first, channels_last

In [ ]:
from pyfacades.models.driving_12x360x480 import process_strip as segment_driving

In [ ]:
from pyfacades.models.independant_12_layers.segment import process_strip

In [ ]:
import pyfacades.models.independant_12_layers as i12

## Put it all together 

In [ ]:
def get_metrics(f, vis=False, rectify=True):
    blob = np.load(f)
    rgb = blob[:3]
    expected = blob[3:]  
    expected_windows = expected[i12.WINDOW]
    
    # Remove padding from considereation
    padding = rgb.max(axis=0).max(axis=0) == 0
    padding = np.outer(np.ones(rgb.shape[1]), padding) > 0
    expected_windows[padding] = 1

    
    
    results = process_strip(rgb)
    predicted_windows = results.features[i12.WINDOW,(0,1)].argmax(axis=0)*2
    
    mf = Metrics(expected=expected_windows, 
             predicted=predicted_windows, 
             label_positive=2,
             label_negative=0,
             source=f,
             feature='window'
            )
    if vis:
        clf()
        subplot(221)
        imshow(channels_last(rgb)/255.)
        axis('off')
        subplot(223)
        imshow(expected_windows, vmin=0, vmax=2)
        axis('off')
        subplot(222)
        imshow(predicted_windows, vmin=0, vmax=2)
        axis('off')
        subplot(224)
        imshow(results.features[i12.WINDOW,1], vmin=0, vmax=1, cmap=cm.gray)
        axis('off')

        
        try:
            suptitle('A:{}, P:{}, R:{}'.format(mf.pixel_accuracy, mf.pixel_precision, mf.pixel_recall))
        except ZeroDivisionError:
            pass

    return mf, rgb, expected_windows, predicted_windows

In [ ]:
figure(figsize=(8,8))
mf, rgb, expected, predicted = get_metrics(eval_files[12], vis=True, rectify=True)

In [ ]:
import anydbm
import json
import hashlib
import munch

In [ ]:
eval_results = anydbm.open('eval_independant_based_on_CMP', 'c')
checksum = hashlib.md5(open(i12.WEIGHTS).read()).hexdigest()

In [ ]:
if 'md5' in eval_results and eval_results['md5'] == checksum:
    print "We already seem to have run evaluation..."

In [ ]:
fig = figure(figsize=(6,6))

recompute = False
visualize = False

total = Metrics(feature='windows')
for i, f in enumerate(eval_files):   
    if recompute or f not in eval_results:
        mf, rgb, expected, predicted = get_metrics(f, visualize)
        eval_results[f] = json.dumps(mf.as_dict())
        if visualize:
            try:
                suptitle('{} of {}, $P$:{:.2%}, $R$:{:.2%}, $F_1$:{:.2%}, $A$:{:.2%}'.format(i, len(eval_files), total.pixel_precision, total.pixel_recall, total.pixel_f1, total.pixel_accuracy))
            except ZeroDivisionError:
                suptitle("Not enough samples yet....")         
            fig.canvas.draw()
    else:
        mf = Metrics(**json.loads(eval_results[f]))
    print mf, 
    total += mf
    print total, "Acc:", total.pixel_accuracy
print "Total(GSV + CMP) &",
print '{:.3f}'.format(total.pixel_accuracy),
print '& {:.2f}'.format(total.pixel_precision),
print '& {:.2f}'.format(total.pixel_recall),
print '& {:.2f}'.format(total.pixel_f1),
print '& {:.2f}'.format(total.object_precision),
print '& {:.2f}'.format(total.object_recall),
print '& {:.2f}'.format(total.object_f1),
print r'\\'

#  The best and the worst

In [ ]:
accuracies = [Metrics(**json.loads(eval_results[f])).pixel_f1 for f in eval_files]
argworst = argsort(accuracies)
worst = array(eval_files)[argworst]
worst_indices = [eval_files.index(k) for k in worst]

In [ ]:
figure()
plot(linspace(0,100, len(accuracies)), array(accuracies)[argworst], label='Multilabel/Edge (CMP)');
ylabel('$F_1$')
xlabel('rank')
legend(loc='best')

In [ ]:
%run -i color_coded_errors.py

In [ ]:
cmp_comparison_files = [fn.strip() for fn in open('cmp_comparison_files.txt')]

import os
fig = figure(figsize=(9,9))
plt.subplots_adjust(wspace=0)
for i in range(9):
    subplot(3,3,i+1)
    cached = 'independant-CMP-top-{}.png'.format(i+1)
    #os.remove(cached)
    if os.path.isfile(cached):
        err_image = imread(cached)
    else:
        err_image = render_errors(cmp_comparison_files[i], alpha=0.6);
        imsave(cached, err_image)
    imshow(err_image)
    xticks([]); yticks([]);
    #xlabel(cached, fontsize=8)
    fig.canvas.draw()
fig.tight_layout()
savefig('independant-CMP-top-9-figure.png', dpi=400)

In [ ]:
for i in range(9):
    cached = 'independant-GSV-top-{}.png'.format(i+1)
    os.remove(cached)

In [ ]:
gsv_comparison_files = [fn.strip() for fn in open('gsv_comparison_files.txt')]

import os
fig = figure(figsize=(9,9))
plt.subplots_adjust(wspace=0)
for i in range(9):
    subplot(3,3,i+1)
    cached = 'independant-GSV-top-{}.png'.format(i+1)
    if os.path.isfile(cached):
        err_image = imread(cached)
    else:
        err_image = render_errors(gsv_comparison_files[i], alpha=0.6);
        imsave(cached, err_image)
    imshow(err_image)
    xticks([]); yticks([]); 
    #xlabel(cached, fontsize=8)
    fig.canvas.draw()
fig.tight_layout()
savefig('independant-GSV-top-9-figure.png', dpi=400)

In [ ]:
figure(figsize=(8, 16))
result = get_metrics(worst[-1])
subplot(321); title('original')
subplot(322); title('label-colors')
subplot(323); title('expected')
subplot(324); title('predicted')
subplot(325); title('rectified')
subplot(326); title('$\Pr[\text{window}]$')
savefig('worst_example.png', dpi=300)

In [ ]:
figure(figsize=(8, 16))
result = get_metrics(worst[0])
subplot(321); title('original')
subplot(322); title('label-colors')
subplot(323); title('expected')
subplot(324); title('predicted')
subplot(325); title('rectified')
subplot(326); title('$\Pr[\text{window}]$')
savefig('worst_example.png', dpi=300)

In [ ]:
figure(figsize=(4, 8))
result = get_metrics(worst[1])
subplot(321); title('original')
subplot(322); title('label-colors')
subplot(323); title('expected')
subplot(324); title('predicted')
subplot(325); title('rectified')
subplot(326); title('$\Pr[\text{window}]$')

In [ ]:
figure(figsize=(4, 8))
result = get_metrics(worst[2])
subplot(321); title('original')
subplot(322); title('label-colors')
subplot(323); title('expected')
subplot(324); title('predicted')
subplot(325); title('rectified')
subplot(326); title('$\Pr[\text{window}]$')

# And on GSV Images

In [ ]:
import os
gsv_total = Metrics()
for f in eval_results:
    if os.path.isfile(f) and '/cmp/' not in f:
        gsv_total  += Metrics(**json.loads(eval_results[f]))
print gsv_total
print 'Acc:', gsv_total.pixel_accuracy

In [ ]:
print "Total(GSV) &",
print '{:.3f}'.format(gsv_total.pixel_accuracy),
print '& {:.2f}'.format(gsv_total.pixel_precision),
print '& {:.2f}'.format(gsv_total.pixel_recall),
print '& {:.2f}'.format(gsv_total.pixel_f1),
print '& {:.2f}'.format(gsv_total.object_precision),
print '& {:.2f}'.format(gsv_total.object_recall),
print '& {:.2f}'.format(gsv_total.object_f1),
print r'\\'